In [1]:
"""
Historical Backtesting
Calculate SPY price change (using the mean value of the price change) after certain trading days when the VIX price above a specific threshold
"""

'\nHistorical Backtesting\nCalculate SPY price change after certain trading days when the VIX price above a specific threshold\n'

In [72]:
# import libraries
import pandas as pd
import Data.DataWrangle as DW
# import datetime
import hvplot.pandas


In [73]:
# import data with shared date
no_NAN = DW.shared_date_data()
no_NAN.head()

,close_SPY,close_TR,close_VIX
timestamp,,,
2015-12-02,208.54,2.7700,15.91
2015-12-03,205.58,4.3989,18.11
2015-12-04,209.66,4.3900,14.81
2015-12-07,208.27,2.5295,15.84
2015-12-08,206.99,2.5090,17.60


In [74]:
# select VIX and SPY data 
working_data = DW.slice_up(['close_SPY', 'close_VIX'])

# summary statistics
vix_tbl = working_data['close_VIX'].describe()
spy_tbl = working_data['close_SPY'].describe()
print(vix_tbl)
print(spy_tbl)

count    1593.000000
mean       18.273503
std         8.207784
min         9.140000
25%        12.830000
50%        16.160000
75%        21.380000
max        82.690000
Name: close_VIX, dtype: float64
count    1593.000000
mean      298.999058
std        76.119125
min       183.030000
25%       243.010000
50%       280.530000
75%       335.840000
max       477.710000
Name: close_SPY, dtype: float64


In [75]:
# working_data.plot(x='timestamp', y= 'close_SPY',)
working_data['close_VIX'].hvplot(label='VIX',xlabel='date', ylabel='price $',title ='VIX and SPY close price from 2015-12-02 to 2022-3-29') *  working_data['close_SPY'].hvplot(label='SPY')

:Overlay
   .Curve.VIX :Curve   [timestamp]   (close_VIX)
   .Curve.SPY :Curve   [timestamp]   (close_SPY)

In [76]:
 working_data.reset_index(inplace=True)
 working_data

,timestamp,close_SPY,close_VIX
0,2015-12-01,210.68,14.67
1,2015-12-02,208.54,15.91
2,2015-12-03,205.58,18.11
3,2015-12-04,209.66,14.81
4,2015-12-07,208.27,15.84
...,...,...,...
1588,2022-03-23,443.80,23.57
1589,2022-03-24,450.48,21.67
1590,2022-03-25,452.69,20.81
1591,2022-03-28,455.91,19.63


In [77]:
# define a function to calculate SPY price change based on VIX value
def backtesting_SPY_price_change_on_VIX(shreshold_1, shreshold_2, days ):
    # get the dataframe for close_VIX is greater a shreshold
    filtered_df =pd.DataFrame(working_data.loc[(working_data['close_VIX']>shreshold_1)& (working_data['close_VIX']<shreshold_2)]).reset_index()
    # rename the columns of the filtered dataframe
    filtered_df.columns =['filtered_index', 'filtered_timestamp', 'close_SPY','close_VIX']
    # get the dataframe for the shifted date 
    shifted_df= working_data.loc[filtered_df['filtered_index'][:-days]+days]
    shifted_df.reset_index(inplace=True)
    # rename the columns of the shifted dataframe
    shifted_df.columns = ['shifted_index','shifted_timestamp','shifted_close_SPY','shifted_close_VIX']
    # combined filtered and shifted dataframes
    combined_df = pd.concat([shifted_df, filtered_df], axis='columns', join='inner')
    # calculate price change
    price_change = combined_df['shifted_close_SPY']-combined_df['close_SPY']
    tbl = price_change.describe()
    if tbl[1] > 0:
        print(f'Historically, when the VIX close price crosses ${shreshold_1} but below ${shreshold_2}, the SPY price increases ${round(tbl[1],2)} on average after {days} trading days.')
    elif tbl[1]< 0:
        print(f'Historically, when the VIX close price crosses ${shreshold_1} but below ${shreshold_2}, the SPY price decreases ${round(-tbl[1],2)} on average after {days} trading days.')
    else:
        print(f'Historically, when the VIX close price crosses ${shreshold_1} but below ${shreshold_2}, the SPY price has no change on average on after {days} trading days.')


    
 

##### When VIX close price is between min and 25% percentile value, SPY close price changes in 28, 100, 252 and 504 trading days.

In [78]:
backtesting_SPY_price_change_on_VIX(round(vix_tbl[3],2),round(vix_tbl[4],2),28)



Historically, when the VIX close price crosses $9.14 but below $12.83, the SPY price increases $2.46 on average after 28 trading days.


In [79]:
backtesting_SPY_price_change_on_VIX(round(vix_tbl[3],2),round(vix_tbl[4],2),100)

Historically, when the VIX close price crosses $9.14 but below $12.83, the SPY price increases $12.72 on average after 100 trading days.


In [80]:
backtesting_SPY_price_change_on_VIX(round(vix_tbl[3],2),round(vix_tbl[4],2),252)

Historically, when the VIX close price crosses $9.14 but below $12.83, the SPY price increases $35.42 on average after 252 trading days.


In [81]:
backtesting_SPY_price_change_on_VIX(round(vix_tbl[3],2),round(vix_tbl[4],2),504)

Historically, when the VIX close price crosses $9.14 but below $12.83, the SPY price has no change on average on after 504 trading days.


##### When VIX close price is between 25% and 50% percentile value, SPY close price changes in 28, 100, 252, and 504 trading days.

In [82]:
backtesting_SPY_price_change_on_VIX(round(vix_tbl[4],2),round(vix_tbl[5],2),28)

Historically, when the VIX close price crosses $12.83 but below $16.16, the SPY price increases $0.79 on average after 28 trading days.


In [83]:
backtesting_SPY_price_change_on_VIX(round(vix_tbl[4],2),round(vix_tbl[5],2),100)

Historically, when the VIX close price crosses $12.83 but below $16.16, the SPY price increases $9.23 on average after 100 trading days.


In [84]:
backtesting_SPY_price_change_on_VIX(round(vix_tbl[4],2),round(vix_tbl[5],2),252)

Historically, when the VIX close price crosses $12.83 but below $16.16, the SPY price increases $32.91 on average after 252 trading days.


In [85]:
backtesting_SPY_price_change_on_VIX(round(vix_tbl[4],2),round(vix_tbl[5],2),504)

Historically, when the VIX close price crosses $12.83 but below $16.16, the SPY price has no change on average on after 504 trading days.


##### When VIX close price is between 50% and 75% percentile value, SPY close price changes in 28, 100, 252, and 504 trading days.

In [86]:
backtesting_SPY_price_change_on_VIX(round(vix_tbl[5],2),round(vix_tbl[6],2),28)

Historically, when the VIX close price crosses $16.16 but below $21.38, the SPY price increases $4.64 on average after 28 trading days.


In [87]:
backtesting_SPY_price_change_on_VIX(round(vix_tbl[5],2),round(vix_tbl[6],2),100)

Historically, when the VIX close price crosses $16.16 but below $21.38, the SPY price increases $22.25 on average after 100 trading days.


In [88]:
backtesting_SPY_price_change_on_VIX(round(vix_tbl[5],2),round(vix_tbl[6],2),252)

Historically, when the VIX close price crosses $16.16 but below $21.38, the SPY price increases $31.29 on average after 252 trading days.


In [89]:
backtesting_SPY_price_change_on_VIX(round(vix_tbl[5],2),round(vix_tbl[6],2),504)

Historically, when the VIX close price crosses $16.16 but below $21.38, the SPY price has no change on average on after 504 trading days.


##### When VIX close price is between 75% percentile and max value, SPY close price changes in 28, 100, 252, and 504 trading days.

In [90]:
backtesting_SPY_price_change_on_VIX(round(vix_tbl[6],2),round(vix_tbl[7],2),28)

Historically, when the VIX close price crosses $21.38 but below $82.69, the SPY price increases $11.4 on average after 28 trading days.


In [91]:
backtesting_SPY_price_change_on_VIX(round(vix_tbl[6],2),round(vix_tbl[7],2),100)

Historically, when the VIX close price crosses $21.38 but below $82.69, the SPY price increases $41.64 on average after 100 trading days.


In [92]:
backtesting_SPY_price_change_on_VIX(round(vix_tbl[6],2),round(vix_tbl[7],2),252)

Historically, when the VIX close price crosses $21.38 but below $82.69, the SPY price increases $80.4 on average after 252 trading days.


In [93]:
backtesting_SPY_price_change_on_VIX(round(vix_tbl[6],2),round(vix_tbl[7],2),504)

Historically, when the VIX close price crosses $21.38 but below $82.69, the SPY price has no change on average on after 504 trading days.
